In [7]:
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\KIM\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [9]:
import os
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score

# 모델 로드
XGB_model = joblib.load(r"C:\Users\KIM\Desktop\bigdataproject\bigdataproject\final\final_final\models-20250120T033752Z-001\models\xgboost_model_with_pca3rd.joblib")
RDF_model = joblib.load(r"C:\Users\KIM\Desktop\bigdataproject\bigdataproject\final\final_final\models-20250120T033752Z-001\models\randomforest_model_with_pca3rd.joblib")

# 기존 학습에서 사용했던 scaler와 PCA 로드
scaler = joblib.load(r"C:\Users\KIM\Desktop\bigdataproject\bigdataproject\final\final_final\models-20250120T033752Z-001\models\scaler_model3rd.joblib")  # 학습 시 저장된 scaler 파일
pca = joblib.load(r"C:\Users\KIM\Desktop\bigdataproject\bigdataproject\final\final_final\models-20250120T033752Z-001\models\pca_model3rd.joblib")  # 학습 시 저장된 PCA 파일

# 메타데이터 추출 함수
def extract_metadata(file_path):
    metadata = {}
    
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        
        # 파일에서 메타데이터 추출
        for line in lines:
            if line.startswith('Date'):
                metadata['Date'] = line.strip().split(',')[1]
            elif line.startswith('Filename'):
                metadata['Filename'] = line.strip().split(',')[1]
            elif line.startswith('Data Label'):
                metadata['Data Label'] = line.strip().split(',')[1]
            elif line.startswith('Label No'):
                metadata['Label No'] = line.strip().split(',')[1]
            elif line.startswith('Motor Spec'):
                motor_spec = line.strip().split(',')[1:]
                metadata['Motor Spec'] = motor_spec
    
    return metadata

# 센서 데이터 추출 함수
def extract_sensor_data(file_path):
    # 파일에서 센서 데이터를 읽기
    data = pd.read_csv(file_path, skiprows=10, header=None)
    data.columns = ['time','Sensor1', 'Sensor2', 'Sensor3','']
    return data

# 피처 계산 함수
def calculate_features(data):
    features = {}

    # 절댓값 평균
    features['Mean'] = np.mean(np.abs(data))

    # 절댓값 최대값
    features['Max'] = np.max(np.abs(data))

    # RMS (Root Mean Square)
    features['RMS'] = np.sqrt(np.mean(data ** 2))

    # Skewness
    features['Skewness'] = pd.Series(data).skew()

    # Kurtosis
    features['Kurtosis'] = pd.Series(data).kurt()

    # Crest Factor
    rms = features['RMS']
    features['Crest Factor'] = features['Max'] / rms if rms != 0 else 0

    # Impulse Factor
    mean = features['Mean']
    features['Impulse Factor'] = features['Max'] / mean if mean != 0 else 0

    # Shape Factor
    features['Shape Factor'] = rms / mean if mean != 0 else 0

    return features

# 데이터 처리 함수
def load_and_process_new_data(file_path, scaler, pca, feature_names=None):
    # 메타데이터 추출
    metadata = extract_metadata(file_path)
    
    # 센서 데이터 추출
    sensor_data = extract_sensor_data(file_path)

    # 각 센서별 피처 계산
    overall_features = {'Label': metadata['Label No']}
    for sensor in ['Sensor1', 'Sensor2', 'Sensor3']:
        features = calculate_features(sensor_data[sensor].values)
        for feature_name, value in features.items():
            overall_features[f'{sensor}_{feature_name}'] = value

    # Motor Spec_Period 추가 (예시: motor spec에서 주기를 계산하여 추가)
    motor_spec_period = 1  # 예시로 1로 설정, 실제 계산 필요
    overall_features['Motor Spec_Period'] = motor_spec_period

    # 피처와 레이블을 데이터프레임으로 변환
    combined_df = pd.DataFrame([overall_features])

    # 피처 (X)와 레이블 (y) 분리
    X = combined_df.drop(columns=['Label'])
    y = combined_df['Label']

    # feature_names에 맞춰서 새 데이터의 순서를 정렬
    if feature_names:
        X = X[feature_names]  # 학습 시 사용한 특성 순서로 정렬

    # 데이터 스케일링
    X_scaled = scaler.transform(X)

    # PCA 적용
    X_pca = pca.transform(X_scaled)

    return X_pca, y

# 디렉토리 내 모든 파일 처리
def process_all_files_in_directory(directory_path, scaler, pca, XGB_model, RDF_model, feature_names=None):
    result = []  # 결과를 저장할 리스트

    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.endswith(".csv"):  # .csv 파일을 처리
                file_path = os.path.join(root, file)
                print(f"Processing file: {file_path}")

                # 새로운 데이터에 대해 처리 및 예측 수행
                X_new_pca, y_new = load_and_process_new_data(file_path, scaler, pca, feature_names)

                # 예측 수행 (XGBOOST 모델)
                XGB_pred = XGB_model.predict(X_new_pca)
                #print(f"XGBOOST Prediction for {file}: {XGB_pred}")

                # 예측 수행 (RANDOMFOREST 모델)
                RDF_pred = RDF_model.predict(X_new_pca)
                #print(f"RandomForest Prediction for {file}: {RDF_pred}")

                # 결과를 리스트에 저장
                result.append({
                    'Filename': file,
                    'XGBOOST Prediction' : XGB_pred[0],
                    'RandomForest Prediction': RDF_pred[0],
                    'True Label': y_new.values[0]
                })
    
    # 결과를 데이터프레임으로 변환
    result_df = pd.DataFrame(result)

    # 예측값 매핑: 0 → 'N', 1 → 'E1', 2 → 'E2', 3 → 'E3', 4 → 'E4'
    label_mapping = {0: 'N', 1: 'E1', 2: 'E2', 3: 'E3', 4: 'E4'}
    result_df['XGB Prediction Label'] = result_df['XGBOOST Prediction'].map(label_mapping)
    result_df['RDF Prediction Label'] = result_df['RandomForest Prediction'].map(label_mapping)

    return result_df

# 불량 유형 비율 계산 함수
def calculate_defect_ratios(result_df):
    for model in ['XGB', 'RDF']:
        print(f"\n{model} Model 비율 계산:")

        total_count = result_df.shape[0]
        
        # 정상(N) 비율 계산
        normal_count = result_df[result_df[f'{model} Prediction Label'] == 'N'].shape[0]
        normal_ratio = normal_count / total_count if total_count > 0 else 0
        print(f"  정상 (N) 비율: {normal_ratio * 100:.2f}%")
        
        # 불량 유형별 비율 계산
        for defect_type in ['E1', 'E2', 'E3', 'E4']:
            defect_count = result_df[result_df[f'{model} Prediction Label'] == defect_type].shape[0]
            defect_ratio = defect_count / total_count if total_count > 0 else 0
            print(f"  {defect_type} 비율: {defect_ratio * 100:.2f}%")

# 학습 데이터에서 특성 이름 추출
feature_names = [
    'Sensor1_Mean', 'Sensor1_Max', 'Sensor1_RMS', 'Sensor1_Skewness', 'Sensor1_Kurtosis', 
    'Sensor1_Crest Factor', 'Sensor1_Impulse Factor', 'Sensor1_Shape Factor', 
    'Sensor2_Mean', 'Sensor2_Max', 'Sensor2_RMS', 'Sensor2_Skewness', 'Sensor2_Kurtosis', 
    'Sensor2_Crest Factor', 'Sensor2_Impulse Factor', 'Sensor2_Shape Factor', 
    'Sensor3_Mean', 'Sensor3_Max', 'Sensor3_RMS', 'Sensor3_Skewness', 'Sensor3_Kurtosis', 
    'Sensor3_Crest Factor', 'Sensor3_Impulse Factor', 'Sensor3_Shape Factor',
    'Motor Spec_Period'  # Motor Spec_Period 추가
]

# 디렉토리 경로
directory_path = r"D:\기계시설물 고장 예지 센서\Training\current\남은파일\L-SF-04\베어링불량"  # 여기에 새로운 데이터 파일이 들어있는 디렉토리 경로를 입력

# 디렉토리 내 모든 파일 처리 및 결과 출력
result_df = process_all_files_in_directory(directory_path, scaler, pca, XGB_model, RDF_model, feature_names=feature_names)
print("\nPrediction Results:")
print(result_df)

# 불량 유형 비율 계산
calculate_defect_ratios(result_df)


KeyError: 'XGBOOST Prediction'

# 시각화 자료

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns
import os

# 한글 폰트 설정 (필요한 경우)
plt.rcParams['font.family'] = 'Malgun Gothic'  # 윈도우
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지

# 저장 경로 설정
output_dir = r"C:\Users\KIM\Desktop\bigdataproject\bigdataproject\final\final_final"  # 저장할 디렉토리 경로
os.makedirs(output_dir, exist_ok=True)  # 디렉토리가 없으면 생성

# 불량 유형 비율 시각화
def visualize_defect_ratios(result_df, output_dir):
    for model in ['XGB', 'RDF']:
        # 불량 유형 비율 계산
        defect_counts = result_df[f'{model} Prediction Label'].value_counts()
        defect_labels = defect_counts.index
        defect_values = defect_counts.values

        # 막대그래프
        plt.figure(figsize=(10, 6))
        sns.barplot(x=defect_labels, y=defect_values, palette="viridis")
        plt.title(f"{model} Model - 예측 결과 분포", fontsize=16)
        plt.xlabel("예측 라벨", fontsize=14)
        plt.ylabel("파일 개수", fontsize=14)
        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)

        # 저장
        bar_output_path = os.path.join(output_dir, f"{model}_Prediction_Bar.jpg")
        plt.savefig(bar_output_path, format="jpg", dpi=300)
        print(f"Saved bar chart: {bar_output_path}")
        plt.close()

        # 파이차트
        plt.figure(figsize=(8, 8))
        plt.pie(defect_values, labels=defect_labels, autopct='%1.1f%%', startangle=140, colors=sns.color_palette("viridis", len(defect_labels)))
        plt.title(f"{model} Model - 예측 결과 비율", fontsize=16)

        # 저장
        pie_output_path = os.path.join(output_dir, f"{model}_Prediction_Pie.jpg")
        plt.savefig(pie_output_path, format="jpg", dpi=300)
        print(f"Saved pie chart: {pie_output_path}")
        plt.close()

# 모델 간 결과 비교 시각화
def visualize_model_comparison(result_df, output_dir):
    # XGBoost vs RandomForest의 예측 결과 비교
    comparison = result_df[['XGB Prediction Label', 'RDF Prediction Label']].value_counts().reset_index()
    comparison.columns = ['XGB Prediction Label', 'RDF Prediction Label', 'Count']

    plt.figure(figsize=(12, 8))
    sns.heatmap(comparison.pivot("XGB Prediction Label", "RDF Prediction Label", "Count"),
                annot=True, fmt='d', cmap='Blues')
    plt.title("XGBoost와 RandomForest 모델의 예측 결과 비교", fontsize=16)
    plt.xlabel("RandomForest Prediction", fontsize=14)
    plt.ylabel("XGBoost Prediction", fontsize=14)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)

    # 저장
    heatmap_output_path = os.path.join(output_dir, "Model_Comparison_Heatmap.jpg")
    plt.savefig(heatmap_output_path, format="jpg", dpi=300)
    print(f"Saved heatmap: {heatmap_output_path}")
    plt.close()

# 함수 실행
visualize_defect_ratios(result_df, output_dir)  # 불량 유형 비율 시각화 및 저장
visualize_model_comparison(result_df, output_dir)  # 모델 간 예측 비교 시각화 및 저장


C:\Users\human\AppData\Local\Temp\ipykernel_6524\1611297124.py:23: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=defect_labels, y=defect_values, palette="viridis")


Saved bar chart: C:\Users\human\Desktop\프로젝트 관련\이미지들\XGB_Prediction_Bar.jpg
Saved pie chart: C:\Users\human\Desktop\프로젝트 관련\이미지들\XGB_Prediction_Pie.jpg


C:\Users\human\AppData\Local\Temp\ipykernel_6524\1611297124.py:23: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=defect_labels, y=defect_values, palette="viridis")


Saved bar chart: C:\Users\human\Desktop\프로젝트 관련\이미지들\RDF_Prediction_Bar.jpg
Saved pie chart: C:\Users\human\Desktop\프로젝트 관련\이미지들\RDF_Prediction_Pie.jpg


TypeError: DataFrame.pivot() takes 1 positional argument but 4 were given

<Figure size 1200x800 with 0 Axes>